# install

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install git+https://github.com/k4black/codebleu.git
!pip install git+https://github.com/k4black/codebleu#egg=codebleu

  Cloning https://github.com/k4black/codebleu.git to /tmp/pip-req-build-5htlmg2g
  Running command git clone --filter=blob:none --quiet https://github.com/k4black/codebleu.git /tmp/pip-req-build-5htlmg2g
  Resolved https://github.com/k4black/codebleu.git to commit ddea467ecbd5cbb2a33e318827740195b08e4aa8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/k4black/codebleu to /tmp/pip-install-h749sbcf/codebleu_0cfb77e81a55478d923499034ce0816b
  Running command git clone --filter=blob:none --quiet https://github.com/k4black/codebleu /tmp/pip-install-h749sbcf/codebleu_0cfb77e81a55478d923499034ce0816b
  Resolved https://github.com/k4black/codebleu to commit ddea467ecbd5cbb2a33e318827740195b08e4aa8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# code

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "EleutherAI/gpt-neo-125m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare model inputs
prompt = "def add_numbers(a, b):\n    \"\"\"Add two numbers.\"\"\"\n    return a + b"
inputs = tokenizer(prompt, return_tensors="pt")


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
!pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:

# Prepare model inputs
prompt = "def add_numbers(a, b):\n    \"\"\"Add two numbers.\"\"\"\n  return a"
inputs = tokenizer(prompt, return_tensors="pt")
print(inputs)
# Call the model to get the logits
outputs = model(**inputs)

# Logits for the last position in the sequence
logits = outputs.logits[0, -1, :]

# Convert logits to probabilities using softmax
probabilities = torch.softmax(logits, dim=-1)

# Get the predicted token ID (the one with the highest probability)
predicted_token_id = torch.argmax(probabilities).item()

# Decode the predicted token ID to text
predicted_token = tokenizer.decode([predicted_token_id])

print(f"Logits: {logits}")
print(f"Probabilities: {probabilities}")
print(f"Predicted token ID: {predicted_token_id}")
print(f"Predicted token: {predicted_token}")

{'input_ids': tensor([[ 4299,   751,    62,    77, 17024,     7,    64,    11,   275,  2599,
           198,   220,   220,   220, 37227,  4550,   734,  3146,   526, 15931,
           198,   220,  1441,   257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Logits: tensor([ -8.8760,  -9.3854, -10.0595,  ..., -27.0245, -22.5152,  -8.9662],
       grad_fn=<SliceBackward0>)
Probabilities: tensor([1.3964e-04, 8.3900e-05, 4.2757e-05,  ..., 1.8331e-12, 1.6656e-10,
        1.2758e-04], grad_fn=<SoftmaxBackward0>)
Predicted token ID: 1343
Predicted token:  +


In [37]:
# Generate top-1 prediction
outputs = model.generate(inputs["input_ids"], num_return_sequences=1)
print(outputs)
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_code)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[ 4299,   751,    62,    77, 17024,     7,    64,    11,   275,  2599,
           198,   220,   220,   220, 37227,  4550,   734,  3146,   526, 15931,
           198,   220,  1441,   257,  1343,   275,   198,   198,  4299,   751,
            62,    77, 17024,     7,    64,    11,   275,  2599,   198,   220,
         37227,  4550,   734,  3146]])
def add_numbers(a, b):
    """Add two numbers."""
  return a + b

def add_numbers(a, b):
  """Add two numbers


In [9]:
from codebleu import calc_codebleu

prediction = generated_code
reference = prompt
print(reference)
print(prediction)
result = calc_codebleu([reference], [prediction], lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
print(result)

def add_numbers(a, b):
    """Add two numbers."""
    return a + b
def add_numbers(a, b):
    """Add two numbers."""
    return a + b

def add_numbers(a, b):
    """Add two numbers
{'codebleu': 0.7707058488538163, 'ngram_match_score': 0.5828233954152653, 'weighted_ngram_match_score': 1.0, 'syntax_match_score': 0.5, 'dataflow_match_score': 1.0}


In [10]:
result

{'codebleu': 0.7707058488538163,
 'ngram_match_score': 0.5828233954152653,
 'weighted_ngram_match_score': 1.0,
 'syntax_match_score': 0.5,
 'dataflow_match_score': 1.0}